# Layer Model

Follows [this section](https://geotrellis.github.io/geotrellis-workshop/docs/layer-model) of the docs.

In [1]:
import $ivy.`org.locationtech.geotrellis::geotrellis-raster:3.5.2`
import $ivy.`org.locationtech.geotrellis::geotrellis-layer:3.5.2`
import $ivy.`org.slf4j:slf4j-simple:1.7.30`
import geotrellis.raster._
import geotrellis.vector._

Downloaded https://repo1.maven.org/maven2/org/locationtech/geotrellis/geotrellis-layer_2.12/3.5.2/geotrellis-layer_2.12-3.5.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/avro/avro/1.7.7/avro-1.7.7.pom
Downloaded https://repo1.maven.org/maven2/jp/ne/opt/chronoscala_2.12/0.3.2/chronoscala_2.12-0.3.2.pom
Downloaded https://repo1.maven.org/maven2/org/scalactic/scalactic_2.12/3.2.1/scalactic_2.12-3.2.1.pom
Downloaded https://repo1.maven.org/maven2/org/apache/avro/avro-parent/1.7.7/avro-parent-1.7.7.pom
Downloaded https://repo1.maven.org/maven2/org/apache/avro/avro-toplevel/1.7.7/avro-toplevel-1.7.7.pom
Downloaded https://repo1.maven.org/maven2/org/apache/apache/10/apache-10.pom
Downloaded https://repo1.maven.org/maven2/org/codehaus/jackson/jackson-core-asl/1.9.13/jackson-core-asl-1.9.13.pom
Downloaded https://repo1.maven.org/maven2/com/thoughtworks/paranamer/paranamer/2.3/paranamer-2.3.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-compress/1.4.1/commo

import $ivy.$                                                     

import $ivy.$                                                    

import $ivy.$                              

import geotrellis.raster._

import geotrellis.vector._

In [2]:
import geotrellis.layer.{ LayoutDefinition, SpatialKey }
import geotrellis.raster.GridExtent
import geotrellis.vector.{ Extent, Point, LineString }

val layout = LayoutDefinition(
  grid = GridExtent(Extent(0, 0, 100, 100), cols = 100, rows = 100),
  tileCols = 10, tileRows = 10
)

import geotrellis.layer.{ LayoutDefinition, SpatialKey }

import geotrellis.raster.GridExtent

import geotrellis.vector.{ Extent, Point, LineString }


layout: LayoutDefinition = LayoutDefinition(
  Extent(0.0, 0.0, 100.0, 100.0),
  TileLayout(10, 10, 10, 10)
)

In [3]:
SpatialKey(0, 9).extent(layout)
SpatialKey(0, 0).extent(layout)

res2_0: Extent = Extent(0.0, 0.0, 10.0, 10.0)
res2_1: Extent = Extent(0.0, 90.0, 10.0, 100.0)

In [4]:
layout.mapTransform.pointToKey(Point(5, 5))

res3: SpatialKey = SpatialKey(0, 9)

In [5]:
layout.mapTransform.pointToKey(Point(15, 15))

res4: SpatialKey = SpatialKey(1, 8)

In [ ]:
layout.mapTransform.keyToExtent(SpatialKey(1,8))

In [ ]:
layout.mapTransform.extentToBounds(Extent(5, 5, 15, 15))

In [6]:
layout.mapTransform.keysForGeometry(LineString(Point(5,5), Point(25,5)))

res5: Set[SpatialKey] = Set(
  SpatialKey(2, 9),
  SpatialKey(1, 9),
  SpatialKey(0, 9)
)

In [7]:
import geotrellis.layer.ZoomedLayoutScheme
import geotrellis.proj4.WebMercator

val scheme = ZoomedLayoutScheme(WebMercator, tileSize = 256)

import geotrellis.layer.ZoomedLayoutScheme

import geotrellis.proj4.WebMercator


scheme: ZoomedLayoutScheme = ZoomedLayoutScheme(WebMercator,256,0.1)

In [8]:
val zoom7 = scheme.levelForZoom(7).layout

zoom7: LayoutDefinition = LayoutDefinition(
  Extent(
    -2.0037508342789244E7,
    -2.0037508342789244E7,
    2.0037508342789244E7,
    2.0037508342789244E7
  ),
  TileLayout(128, 128, 256, 256)
)

In [9]:
val zoom8 = scheme.levelForZoom(8).layout

zoom8: LayoutDefinition = LayoutDefinition(
  Extent(
    -2.0037508342789244E7,
    -2.0037508342789244E7,
    2.0037508342789244E7,
    2.0037508342789244E7
  ),
  TileLayout(256, 256, 256, 256)
)

In [10]:
val ottawaWM = Extent(-8621691, 5604373, -8336168, 5805297)

ottawaWM: Extent = Extent(-8621691.0, 5604373.0, -8336168.0, 5805297.0)

In [23]:
val tileBounds = zoom7.mapTransform.extentToBounds(ottawaWM)

tileBounds: geotrellis.layer.package.TileBounds = GridBounds(36, 45, 37, 46)

In [21]:
val tileExtent = zoom7.mapTransform.boundsToExtent(tileBounds)

tileExtent: Extent = Extent(
  -8766409.899970295,
  5322463.153553393,
  -8140237.7642581295,
  5948635.289265556
)

In [24]:
// Note that this is snapping out extent to closest tile borders
tileExtent.contains(ottawaWM)
// res9: Boolean = true
ottawaWM.contains(tileExtent)
// res10: Boolean = false

res23_0: Boolean = true
res23_1: Boolean = false

In [35]:
import geotrellis.raster._
import geotrellis.raster.geotiff._
import geotrellis.layer._

val uri1 = "https://geotrellis-workshop.s3.amazonaws.com/landsat/LC81070362015218LGN00_B3.TIF"

val rs1 = GeoTiffRasterSource(uri2)

val lts = rs1.tileToLayout(zoom7)

lts.keys

lts.readAll().take(1).foreach { case (key, tile) =>
    println(s"key: ${key}")
    Image(tile.band(0).renderPng())
}


key: SpatialKey(64,51)


import geotrellis.raster._

import geotrellis.raster.geotiff._

import geotrellis.layer._


uri1: String = "https://geotrellis-workshop.s3.amazonaws.com/landsat/LC81070362015218LGN00_B3.TIF"
rs1: GeoTiffRasterSource = GeoTiffRasterSource(https://geotrellis-workshop.s3.amazonaws.com/landsat/LC81070362015218LGN00_B3.TIF)
lts: LayoutTileSource[SpatialKey] = geotrellis.layer.LayoutTileSource@5193773b
res34_6: Set[SpatialKey] = Set(
  SpatialKey(64, 51),
  SpatialKey(65, 51),
  SpatialKey(65, 52),
  SpatialKey(64, 52)
)